In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
!pip install langchain
!pip install chromadb
!pip install openai
!pip install tiktoken
!pip install gpt4all

In [ ]:
!pip install huggingface_hub

# Process document

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import GPT4AllEmbeddings
from langchain.llms import GPT4All
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain import HuggingFaceHub
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from chromadb.errors import InvalidDimensionException
import json

# Load and process the text
loader = TextLoader('/content/p1_whispercppb9e5770b6ce7086e25c131b33f1c9bfc1b0b45db823137f9a0c1d32c46089253.txt')
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

try:
    db = Chroma.from_documents(texts, GPT4AllEmbeddings())
except InvalidDimensionException:
    Chroma().delete_collection()
    db = Chroma.from_documents(texts, GPT4AllEmbeddings())

docs = db.similarity_search("Summarize the conversation!", k=5)
print(docs)

Found model file at  /root/.cache/gpt4all/ggml-all-MiniLM-L6-v2-f16.bin
[Document(page_content="[00:54:16.000 --> 00:54:22.000]   Or do you think there's anything you missed out sharing?\n[00:54:22.000 --> 00:54:26.000]   I think I covered most of it.\n[00:54:26.000 --> 00:54:29.000]   Yes, I think so too.\n[00:54:29.000 --> 00:54:30.000]   Yeah.\n[00:54:30.000 --> 00:54:31.000]   Okay, thank you.\n[00:54:31.000 --> 00:54:32.000]   I'll take the six.\n[00:54:32.000 --> 00:54:34.000]   Anything else to say?\n[00:54:34.000 --> 00:54:36.000]   No, I'll take the six.\n[00:54:36.000 --> 00:54:38.000]   Let me stop the recording.", metadata={'source': '/content/p1_whispercppb9e5770b6ce7086e25c131b33f1c9bfc1b0b45db823137f9a0c1d32c46089253.txt'}), Document(page_content='[00:47:26.000 --> 00:47:35.000]   Like, "Oh, you know, because he\'s a man, he\'s a male, so definitely it\'s easier for him to climb up the ladder."\n[00:47:35.000 --> 00:47:47.000]   I don\'t think so though. I think it makes

# QA over document

In [ ]:
!pip install --upgrade torch transformers einops accelerate
!pip install --upgrade accelerate

In [ ]:
# from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
# callbacks = [StreamingStdOutCallbackHandler()]


# # llm = GPT4All(model="/content/ggml-mpt-7b-instruct.bin",callbacks=callbacks, verbose=True)
# # llm

# llm = HuggingFaceHub(
#     repo_id="mosaicml/mpt-7b-instruct", model_kwargs={"temperature": 0.5, "max_length": 64}
# )

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from transformers import BitsAndBytesConfig
import torch

nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained("mosaicml/mpt-7b-instruct", quantization_config=nf4_config)

In [ ]:
%%time

def transcript_qa(prompt):
  qa = RetrievalQA.from_chain_type(
      llm=llm,
      chain_type="refine",
      retriever=db.as_retriever()
      )
  completion =qa.run(prompt)
  print("Prompt: " + prompt)
  print("Completion: "+ completion + "\n")

transcript_qa("Summarize the conversation!")

transcript_qa("How many speaker are there in this conversation? and how do you know it?")
